In [1]:



# 6. Mostrar os resultados
for i, y in enumerate(Y_pred):
    print(f"Entrada {i+1}: Reynolds = {X_novo[i, 0]}, Porosidade = {X_novo[i, 1]} -> Previsão Y = {y[0]:.4f}")


Entrada 1: Reynolds = 1200.0, Porosidade = 0.3 -> Previsão Y = 0.6839
Entrada 2: Reynolds = 3500.0, Porosidade = 0.6 -> Previsão Y = 0.7420


In [2]:
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
import pandas as pd
import joblib

# 1. Carregar o modelo e os scalers
mlp_model = joblib.load('mlp_model_14_12_2024_loocv.pkl')       # Modelo treinado
scalerx = joblib.load('scalerx_mlp_model_14_12_2024loocv.pkl')  # Scaler de entrada
scalery = joblib.load('scalery_mlp_model_14_12_2024loocv.pkl')  # Scaler de saída

# 2. Definir os intervalos para Re e Stdp
Re_vals = np.linspace(200, 8000, 100)  # Valores de Reynolds
Stdp_vals = np.linspace(0.2992, 0.4331, 100)  # Valores de Stdp (Porosidade)

# 3. Criar uma grade de Re e Stdp
Re_grid, Stdp_grid = np.meshgrid(Re_vals, Stdp_vals)
grid_points = np.c_[Re_grid.ravel(), Stdp_grid.ravel()]

# 4. Previsão com a ANN nos pontos da grade
f_pred_scaled = mlp_model.predict(scalerx.transform(grid_points))
f_pred = scalery.inverse_transform(f_pred_scaled.reshape(-1, 1))
f_pred_grid = f_pred.reshape(Re_grid.shape)

# 5. Calcular a superfície usando a fórmula fornecida
f_formula = (4.3059 / np.log(Re_grid + 3.1918)) + ((0.30649 / Stdp_grid) - 0.80937)

# 6. Carregar dados originais de validação (exemplo: ANNDF)
file_name = 'fxReynoldsV3.xlsx'
ANNDF = pd.read_excel(file_name)

# 7. Gerar o gráfico

# Superfície da ANN
surface_ann = go.Surface(
    x=Re_grid, y=Stdp_grid, z=f_pred_grid, 
    colorscale='Viridis',
    name='Superfície ANN',
    colorbar=dict(title='ANN f', len=0.5, x=0.9)
)

# Superfície da Fórmula
surface_formula = go.Surface(
    x=Re_grid, y=Stdp_grid, z=f_formula, 
    colorscale='Plasma',
    name='Superfície Fórmula',
    opacity=0.7,
    colorbar=dict(title='Fórmula f', len=0.5, x=1.05)
)

# Scatter plot dos valores originais
scatter_original = go.Scatter3d(
    x=ANNDF.iloc[:, 1],  # Reynolds
    y=ANNDF.iloc[:, 3],  # Stdp
    z=ANNDF.iloc[:, 2],  # f
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Dados Originais'
)

# Scatter plot de validação
scatter_validation = go.Scatter3d(
    x=grid_points[:, 0],  # Reynolds
    y=grid_points[:, 1],  # Stdp
    z=f_pred,             # Valores previstos
    mode='markers',
    marker=dict(size=4, color='blue', symbol='diamond'),
    name='Validação ANN'
)

# Layout
layout = go.Layout(
    title='Superfície de Resultados da ANN, Fórmula e Dados Originais',
    scene=dict(
        xaxis=dict(title='Reynolds Number (Re)', backgroundcolor='white', gridcolor='lightgrey'),
        yaxis=dict(title='Stdp', backgroundcolor='white', gridcolor='lightgrey'),
        zaxis=dict(title='f', backgroundcolor='white', gridcolor='lightgrey')
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=True
)

# Criar a figura com ambas as superfícies e os dados
fig = go.Figure(data=[surface_ann, surface_formula, scatter_original, scatter_validation], layout=layout)

# Mostrar o gráfico
pyo.plot(fig)


'temp-plot.html'